### Загружаем данные

* Данные по кампаниям за 1 год
* **5369** кампаний
* **64** параметра
* Параметры рассчитывались за **0,1,2 и 3 дни** жизни пользователя
* Нужно прогнозировать **CARPU 60 дня**

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('data_campaigns.csv',delimiter =',')
data.head()

In [ ]:
data.describe()

In [ ]:
# Исключим мелкие когорты (< 75 квантили)(А что будет, если не будем исключать?)
cohort_size=data["size"].quantile(0.75)
data=data[data['size']>=cohort_size].reset_index(drop=True)
data.describe()

### (!) Пропущенные значения

In [ ]:
#Сколько у нас пропущенных значений?
missing = (data.isnull().sum() / len(data)) * 100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing = pd.DataFrame({'Missing Ratio': missing})
missing.head(20)

In [ ]:
# Заполним пропущенные значения и проверим
data.fillna(0, inplace = True)

missing = (data.isnull().sum() / len(data)) * 100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing = pd.DataFrame({'Missing Ratio': missing})
missing.head()

In [ ]:
# Зададим X и y
X = data.iloc[:,1:65]
y = data['y_target']
y.describe()

### (!) Нормализация (трансформация Бокса-Кокса), коэффициент асимметрии и логарифмирование

In [ ]:
# Нормализуем X трансформацией Бокса-Кокса
from scipy.special import boxcox1p
from scipy.stats import skew
import numpy as np

numeric_feats = X.dtypes[X.dtypes!="object"].index
skewed_feats = X[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending = False)
skewness = pd.DataFrame({'Skew':skewed_feats})
skewness = skewness[abs(skewness)>0.2]
skewed_features = skewness.index
lam=0.04
for feat in skewed_features:
    X[feat] = boxcox1p(X[feat],lam)

# Логарифмируем target и смотрим коэффициент асимметрии. Для реальных результатов незабываем потенциировать
y_n=boxcox1p(y,0)
print("Коэффициент ассиметрии y до транфсормации – %s, b и после – %s"% (skew(y),skew(y_n)))


In [ ]:
y_n.describe()

### (!) Robust и Standard Scaling

In [ ]:
# Масштабируем с Robust Scaler-ом (median/IQR, а не mean/variance)
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()
names = list(X.columns)
X= sc.fit_transform(X)
X = pd.DataFrame(data=X, columns=names)
X.fillna(0, inplace = True)

In [ ]:
X.head()

### Перейдем к моделированию
* **(!) Модель:** Будем использовать Ridge regression
* **(!) Кросс-валидация**: алгоритм ShuffleSplit
* **Работа с параметрами:** Pearson score, RFECV
* **Регуляризация**. Посмотрим как выбирается параметр Alpha
* **Learning Curve**: Посмотрим как ведет себя модель на обучающей и валидирующей выборках
* **Residuals Plot**: Посмотрим как ведут себя остатки (разница между реальным и пронозным значениями)
* **Prediction Error Plot**: Сравним прогнозные результаты модели с ее реальными значениями

In [ ]:
from sklearn.linear_model import  RidgeCV
from sklearn.model_selection import ShuffleSplit, train_test_split


cv=ShuffleSplit(n_splits=10,train_size=0.8,test_size=0.2,random_state= 2019)
alphas = np.logspace(-4, 1, 50)
regressor = RidgeCV(alphas=alphas)

### (!) Смотрим R2 и MSE + yellowbrick framework

In [ ]:
import matplotlib.pyplot as plt
from yellowbrick.model_selection import CVScores

_, ax = plt.subplots(figsize=(10,10))
r2 = CVScores(regressor,ax=ax,cv=cv,scoring='r2')
r2.fit(X,y_n)
r2.poof()

In [ ]:
_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring='neg_mean_squared_error')
mse.fit(X,y_n)
mse.poof()

In [ ]:
# посмотрим как ведет себя RMSE
from sklearn.metrics import make_scorer
def rmse(y, y_pred):
    return np.sqrt(np.mean((y_pred - y)**2))

scorer = make_scorer(rmse,greater_is_better=False)

_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring=scorer)
mse.fit(X,y_n)
mse.poof()

In [ ]:
y_n.describe()

### (!) Посмотрим на возможные выбросы (обычный МНК, расстояние Кука)

In [ ]:
from statsmodels.formula.api import OLS

m = OLS(y_n,X).fit()
infl = m.get_influence()
sm_fr = infl.summary_frame()
_, ax = plt.subplots(figsize=(20,10))
plt.plot(sm_fr['cooks_d'])
plt.show()

In [ ]:
X_out=X.drop(sm_fr['cooks_d'][sm_fr['cooks_d']>4/len(X)].index)
y_out=y_n.drop(sm_fr['cooks_d'][sm_fr['cooks_d']>4/len(X)].index)

In [ ]:
m = OLS(y_out,X_out).fit()
infl = m.get_influence()
sm_fr = infl.summary_frame()
_, ax = plt.subplots(figsize=(20,10))
plt.plot(sm_fr['cooks_d'])
plt.show()

In [ ]:
X_out=X_out.drop(sm_fr['cooks_d'][sm_fr['cooks_d']>4/len(X)].index)
y_out=y_out.drop(sm_fr['cooks_d'][sm_fr['cooks_d']>4/len(X)].index)

In [ ]:
m = OLS(y_out,X_out).fit()
infl = m.get_influence()
sm_fr = infl.summary_frame()
_, ax = plt.subplots(figsize=(20,10))
plt.plot(sm_fr['cooks_d'])
plt.show()

In [ ]:
X_out.shape

### (!) Поработаем с параметрами (корреляционная матрица)

In [ ]:
# Посмотрим коллеряционную матрицу
from yellowbrick.features import Rank2D,RFECV
_, ax = plt.subplots(figsize=(20,10))
rank = Rank2D(ax=ax,features=names,algorithm='pearson')
rank.fit(X_out,y_out)
rank.transform(X)
rank.poof()

### (!) Recursive Feature Elimination

In [ ]:
%%time
# Попробуем посмотреть Recursive Feature Elimination
_, ax = plt.subplots(figsize=(20,10))
rfe = RFECV(regressor,cv=cv,scoring=scorer,step=10)
rfe.fit(X_out,y_out)
rfe.poof()

In [ ]:
#Сохраним выбранные параметры
params = {
    'parameters': list(X.columns),
    'ranking':list(rfe.ranking_)
}
par = pd.DataFrame(data=params)
huber_params = list(par[par['ranking']==1]['parameters'])

In [ ]:
# Посмотрим корреляционную матрицу еще раз
_, ax = plt.subplots(figsize=(20,10))
rank = Rank2D(ax=ax,features=huber_params,algorithm='pearson')
rank.fit(X_out[huber_params],y_n)
rank.transform(X_out[huber_params])
rank.poof()

### (!) Регуляризация

In [ ]:
from yellowbrick.regressor import AlphaSelection
_, ax = plt.subplots(figsize=(20,10))
a_select=AlphaSelection(RidgeCV(alphas=alphas))
a_select.fit(X_out[huber_params],y_out)
a_select.poof()

### Посмотрим Learning Curve

In [ ]:
from yellowbrick.model_selection import LearningCurve
sizes = np.linspace(0.2, 1.0, 5)
_, ax = plt.subplots(figsize=(20,10))
lc = LearningCurve(RidgeCV(alphas=alphas), scoring=scorer,cv=cv)
lc.fit(X_out[huber_params],y_out)
lc.poof()

In [ ]:
_, ax = plt.subplots(figsize=(20,10))
lc = LearningCurve(RidgeCV(alphas=alphas), scoring='r2',cv=cv)
lc.fit(X_out[huber_params],y_out)
lc.poof()

### Посмотрим, что нам дает обученная модель

In [ ]:
# Смотрим как ведут себя остатки
from sklearn.model_selection import train_test_split
from yellowbrick.regressor import ResidualsPlot,PredictionError
X_train,X_test,y_train,y_test = train_test_split(X_out[huber_params],y_out,test_size=0.2,random_state=2019)
regressor=RidgeCV(alphas=alphas,cv=cv,scoring=scorer)

_, ax = plt.subplots(figsize=(20,10))
res = ResidualsPlot(regressor)
res.fit(X_train,y_train)
res.score(X_test,y_test)
res.poof()

In [ ]:
# Смотрим как распределены ошибки
_, ax = plt.subplots(figsize=(20,10))
error=PredictionError(regressor)
error.fit(X_train,y_train)
error.score(X_test,y_test)
error.poof()

In [ ]:
_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring=scorer)
mse.fit(X_out[huber_params],y_out)
mse.poof()

### В итоге
* R2 ≈ **?**
* ln(RMSE) ≈ **?**